# MNIST digit classifier - test reproducibility

In this notebook, we test reproducibility of training by:
- Training a network on a small subset of the MNIST dataset and getting the values of its parameters
- Training a second network using the same initial conditions and getting the values of its parameters, then ensuring taht the parameter values match between the two networks
- Restoring the state of the parameters of the first network to their initial values then training it again. The resulting parameter values are acquired and compared against the parameter values of the first training run.

Turn of cuDNN as its convolution operations are *not reproducible*.

In [1]:
import os
os.environ['THEANO_FLAGS'] = 'dnn.enabled=False, optimizer_including='

#### Imports

In [2]:
import numpy as np

import collections

from matplotlib import pyplot as plt

import sklearn.cross_validation

import lasagne
import theano
import theano.tensor as T
from britefury_lasagne import basic_dnn, trainer, image_window_extractor, mnist

/Users/Geoff/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Define network architecture

We define the `build_network` function that takes the input variables as an optional argument and build the network using the Lasagne API.

NOTE that the final dense layer does *NOT* use the `softmax` nonlinearity as it is supplied by the classifier builder (see below).

In [3]:
def build_network(input_vars=None):
    # Input layer
    x_var = input_vars[0] if input_vars is not None else None
    net = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=x_var)

    # A 32 unit 5x5 conv layer, followed by 2x2 max-pool
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(5, 5), W=lasagne.init.HeUniform(), name='c1_1')
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2, 2), name='p1')

    # Two 32 unit 3x3 conv layers, followed by 2x2 max-pool
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3, 3), W=lasagne.init.HeUniform(), name='c2_1')
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3, 3), W=lasagne.init.HeUniform(), name='c2_2')
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2, 2), name='p2')

    # A fully-connected layer of 64 units
    net = lasagne.layers.DenseLayer(net, num_units=64, W=lasagne.init.HeUniform(), name='d3')
    
    # NO DROPOUT; dropout state is difficult to reset/restore in such a way as to ensure reproducibility;
    # Even `dropout_layer._srng.set_rstate(some_constant)` does not seem to work
    # net = lasagne.layers.DropoutLayer(net, p=0.5)

    # Final 10-unit dense layer, with no nonlinearity
    net = lasagne.layers.DenseLayer(net, num_units=10, nonlinearity=None, name='d4')

    return net

#### Load the train, validation and test sets

In [4]:
dataset = mnist.MNIST()

# Select a small subset. We are testing for reproducibility of computations;
# classifier accuracy is not really important.
train_X = dataset.train_X[:512]
train_y = dataset.train_y[:512]
val_X = dataset.val_X[:512]
val_y = dataset.val_y[:512]
test_X = dataset.test_X[:512]
test_y = dataset.test_y[:512]

### Train the classifier

In [5]:
# Create SEPARATE random number generators for generating weights and shuffling,
# with specific seeds so that we can re-try the experiment below to check that we get the same result
# Creating a network's layers draws from an RNG in order to create randomly initialised network weights.
# Since the third experiment does not create new layers, we need separate RNGs for the shuffling
# step so that we can ensure that it operates the same way each time
weight_rng1 = np.random.RandomState(12345)
shuffle_rng1 = np.random.RandomState(24680)

# Set Lasagne's RNG
lasagne.random.set_rng(weight_rng1)

# Build the image classifier for the given model builder
print('Building network')
clf = basic_dnn.simple_classifier(build_network, n_input_spatial_dims=2,
            updates_fn=lambda loss, params: lasagne.updates.adam(loss, params, learning_rate=0.001))

# Get the randomly initialised parameter values so that we can try to re-start training without building
# the net from scratch
blank_state = clf.get_param_values(include_updates=True)

# Train with a batch size of 128, for 5 epochs, reporting after each epoch,
# using a provided random number generator for shuffling to ensure reproducibility
print('Training')
clf.train([train_X, train_y], [val_X, val_y], [test_X, test_y],
          batchsize=128, num_epochs=5, verbosity=trainer.VERBOSITY_EPOCH,
          shuffle_rng=shuffle_rng1)

clf_state = clf.get_param_values(include_updates=True)

Building network
Training
Epoch 1 took 0.94s:  TRAIN y loss=2.219832  VAL y loss=2.083671 err=53.71%  TEST y loss=2.079519 err=52.73%
Epoch 2 took 0.95s:  TRAIN y loss=1.914864  VAL y loss=1.759531 err=45.90%  TEST y loss=1.747002 err=45.12%
Epoch 3 took 1.01s:  TRAIN y loss=1.489491  VAL y loss=1.341996 err=34.96%  TEST y loss=1.317358 err=34.57%
Epoch 4 took 1.02s:  TRAIN y loss=1.014338  VAL y loss=0.995141 err=28.71%  TEST y loss=0.964328 err=29.30%
Epoch 5 took 0.89s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%
Best result:
Epoch 5 took 4.82s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%


Once more:

In [6]:
weight_rng2 = np.random.RandomState(12345)
shuffle_rng2 = np.random.RandomState(24680)

lasagne.random.set_rng(weight_rng2)

# Build the image classifier for the given model builder
print('Building network')
clf2 = basic_dnn.simple_classifier(build_network, n_input_spatial_dims=2,
            updates_fn=lambda loss, params: lasagne.updates.adam(loss, params, learning_rate=0.001))

# Train
print('Training')
clf2.train([train_X, train_y], [val_X, val_y], [test_X, test_y],
           batchsize=128, num_epochs=5, verbosity=trainer.VERBOSITY_EPOCH,
           shuffle_rng=shuffle_rng2)

clf2_state = clf2.get_param_values(include_updates=True)

Building network
Training
Epoch 1 took 1.00s:  TRAIN y loss=2.219832  VAL y loss=2.083671 err=53.71%  TEST y loss=2.079519 err=52.73%
Epoch 2 took 0.99s:  TRAIN y loss=1.914864  VAL y loss=1.759531 err=45.90%  TEST y loss=1.747002 err=45.12%
Epoch 3 took 0.92s:  TRAIN y loss=1.489491  VAL y loss=1.341996 err=34.96%  TEST y loss=1.317358 err=34.57%
Epoch 4 took 0.81s:  TRAIN y loss=1.014338  VAL y loss=0.995141 err=28.71%  TEST y loss=0.964328 err=29.30%
Epoch 5 took 0.79s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%
Best result:
Epoch 5 took 4.50s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%


Reset the state of the first classifier and train again:

In [7]:
shuffle_rng3 = np.random.RandomState(24680)

# Reset parameter state
clf.set_param_values(blank_state, include_updates=True)

# Train
print('Training')
clf.train([train_X, train_y], [val_X, val_y], [test_X, test_y],
          batchsize=128, num_epochs=5, verbosity=trainer.VERBOSITY_EPOCH,
          shuffle_rng=shuffle_rng3)

clf_state_b = clf.get_param_values(include_updates=True)

Training
Epoch 1 took 0.81s:  TRAIN y loss=2.219832  VAL y loss=2.083671 err=53.71%  TEST y loss=2.079519 err=52.73%
Epoch 2 took 0.89s:  TRAIN y loss=1.914864  VAL y loss=1.759531 err=45.90%  TEST y loss=1.747002 err=45.12%
Epoch 3 took 0.98s:  TRAIN y loss=1.489491  VAL y loss=1.341996 err=34.96%  TEST y loss=1.317358 err=34.57%
Epoch 4 took 0.91s:  TRAIN y loss=1.014338  VAL y loss=0.995141 err=28.71%  TEST y loss=0.964328 err=29.30%
Epoch 5 took 0.79s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%
Best result:
Epoch 5 took 4.39s:  TRAIN y loss=0.671612  VAL y loss=0.845743 err=26.95%  TEST y loss=0.780863 err=27.54%


Check parameters for equality:

In [8]:
def compare_states(s1, s2):
    success = True
    for i, (a, b) in enumerate(zip(s1, s2)):
        if (a != b).any():
            print('FAIL at index {}/{}'.format(i, len(s1)))
            success = False
    if success:
        print('States are identical')
            


In [9]:
compare_states(clf_state, clf2_state)

States are identical


In [10]:
compare_states(clf_state, clf_state_b)

States are identical
